In [ ]:
import pandas as pd
from glob import glob

<h1> 切割照片
<h5> 切完後的路經相同 Cpath 路徑 (第三個cell) </h5>

In [ ]:
filelist = glob(r"E:\nopeople_machine\felic\crop\*csv")
count =0
for item in filelist:
    dellist = []
    file = pd.read_csv(item,header = 0,names = ["id","x","y","w","h","conf","IDcode"])
    Fname = item.split("\\")[-1]
    cn = Fname.split(".")[0]
    print(cn)
    image = cv2.imread(r"E:\nopeople_machine\public"+ "\\"+ cn + ".png")
    (imgheight,imgwidth,rgb) = image.shape
    for i in range(len(file["id"])):
        #print(imgwidth)
        if imgwidth == 1920:
            if (file["IDcode"][i] == "3_1") and ((int(file["x"][i]) ) < 960):
                dellist.append(i)
            elif (file["IDcode"][i] == "2_1") and ((int(file["x"][i]) + int(file["w"][i]))> 1376) :
                dellist.append(i)
            elif (file["IDcode"][i] == "2_1") and  (int(file["x"][i]) < 544) :
                dellist.append(i)
            elif (file["IDcode"][i] == "1_1") and  ((int(file["x"][i]) + int(file["w"][i]))> 960) :
                dellist.append(i)
        else:
            if (file["IDcode"][i] == "3_1") and (int(file["x"][i]) < 384):
                dellist.append(i)
            elif (file["IDcode"][i] == "2_1") and ((int(file["x"][i])+ int(file["w"][i])) > 1088) :
                dellist.append(i)
            elif (file["IDcode"][i] == "2_1") and  (int(file["x"][i]) < 256) :
                dellist.append(i)
            elif (file["IDcode"][i] == "1_1") and  ((int(file["x"][i]) + int(file["w"][i])) > 960) :
                dellist.append(i)
    count += len(file["id"])
    file.drop(file.index[dellist],axis = 0,inplace = True)
    file.to_csv(r"E:\nopeople_machine\felic\new" + "\\" + cn + ".csv" ,index = 0) 

        

<h1> IOU
<h5> 存檔路徑跟第五個cell file路徑相同 </h5>

In [ ]:
import torch
def bbox_iou(box1, box2, x1y1x2y2=True, GIoU=False, DIoU=False, CIoU=False, eps=1e-7):
    # Returns the IoU of box1 to box2. box1 is 4, box2 is nx4
    box2 = box2.T
    
    # Get the coordinates of bounding boxes
    if x1y1x2y2:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[0], box1[1], box1[2], box1[3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[0], box2[1], box2[2], box2[3]
    else:  # transform from xywh to xyxy
        b1_x1, b1_x2 = box1[0] - box1[2] / 2, box1[0] + box1[2] / 2
        b1_y1, b1_y2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
        b2_x1, b2_x2 = box2[0] - box2[2] / 2, box2[0] + box2[2] / 2
        b2_y1, b2_y2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2

    # Intersection area
    inter = (torch.min(b1_x2, b2_x2) - torch.max(b1_x1, b2_x1)).clamp(0) * \
            (torch.min(b1_y2, b2_y2) - torch.max(b1_y1, b2_y1)).clamp(0)

    # Union Area
    w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1 + eps
    w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1 + eps
    union = w1 * h1 + w2 * h2 - inter + eps

    iou = inter / union

    if GIoU or DIoU or CIoU:
        cw = torch.max(b1_x2, b2_x2) - torch.min(b1_x1, b2_x1)  # convex (smallest enclosing box) width
        ch = torch.max(b1_y2, b2_y2) - torch.min(b1_y1, b2_y1)  # convex height
        if CIoU or DIoU:  # Distance or Complete IoU https://arxiv.org/abs/1911.08287v1
            c2 = cw ** 2 + ch ** 2 + eps  # convex diagonal squared
            rho2 = ((b2_x1 + b2_x2 - b1_x1 - b1_x2) ** 2 +
                    (b2_y1 + b2_y2 - b1_y1 - b1_y2) ** 2) / 4  # center distance squared
            if DIoU:
                return iou - rho2 / c2  # DIoU
            elif CIoU:  # https://github.com/Zzh-tju/DIoU-SSD-pytorch/blob/master/utils/box/box_utils.py#L47
                v = (4 / math.pi ** 2) * torch.pow(torch.atan(w2 / (h2 + eps)) - torch.atan(w1 / (h1 + eps)), 2)
                with torch.no_grad():
                    alpha = v / (v - iou + (1 + eps))
                return iou - (rho2 / c2 + v * alpha)  # CIoU
        else:  # GIoU https://arxiv.org/pdf/1902.09630.pdf
            c_area = cw * ch + eps  # convex area
            return iou - (c_area - union) / c_area  # GIoU
    else:
        return iou  # IoU
def countIOU(text,Fname):
    del_index = []                                                                                                                   # correcting the index that need to remove
    del_dis = []  
    same = []                                                                                                                   # correcting the distance that need to remove (it's doesn't used)
    for i in range(len(text)):
        for j in range(i,len(text)):
            if (i != j) and (text["IDcode"][i] != text["IDcode"][j]) and (text["id"][i] == text["id"][j]): 
                box1 =torch.tensor([text["x"][i],text["y"][i],text["x"][i]+text["w"][i],text["y"][i]+text["h"][i]])
                box2 =torch.tensor([text["x"][j],text["y"][j],text["x"][j]+text["w"][j],text["y"][j]+text["h"][j]])                 # Comparing the IDcode , id is the same or not and let i not equal to j
                #iou = IOU(text["x"][i],text["y"][i],text["w"][i],text["h"][i],text["x"][j],text["y"][j],text["w"][j],text["h"][j])   # Call the IOU function
                iou = bbox_iou(box1,box2)
                if iou >= 0.5:  
                    iou = iou.item()                                                                                                     # if IOU <= 0.5 ,del
                    if text["conf"][i] > text["conf"][j]:                                                                            # Comparing the confidance which is larger                 
                        del_index.append(j)
                        del_dis.append([j,i,text["IDcode"][i],text["IDcode"][j],text["id"][i],text["id"][j],iou])
                    elif text["conf"][j] > text["conf"][i]:
                        del_index.append(i)
                        del_dis.append([i,j,text["IDcode"][i],text["IDcode"][j],text["id"][i],text["id"][j],iou])
                    else:
                        iarea = text["w"][i]*text["h"][i]
                        jarea = text["w"][j]*text["h"][j]
                        area = min([iarea,jarea])
                        if area == iarea:
                            if iarea != jarea:
                                del_index.append(j)
                                del_dis.append([j,i,text["IDcode"][i],text["IDcode"][j],text["id"][i],text["id"][j],iou])
                            else:
                                if (i or j)  not in del_index:
                                 del_index.append(j)   
                        else:
                            if iarea != jarea:
                                del_index.append(i)
                                del_dis.append([i,j,text["IDcode"][i],text["IDcode"][j],text["id"][i],text["id"][j],iou])
                            else:
                                if (i or j)  not in del_index:
                                 del_index.append(j) 

                    iarea = text["w"][i]*text["h"][i]
                    jarea = text["w"][j]*text["h"][j]
                    if iarea != jarea :
                        if text["conf"][i] >= text["conf"][j]:
                            same.append(j)
                        else:
                            same.append(i)

    same = sorted(list(set(same)))                    
    del_index = sorted(list(set(del_index)))
    #del_dis = list(set(del_dis))
    text.drop(text.index[del_index],axis = 0,inplace = True)
    text.to_csv(r"E:\nopeople_machine\nms\new" + "\\" + Fname ,index = 0)                                                                             # del the index
    return del_index,same                                                                                                                 # return index which has been del            
    text.to_csv(r"E:\nopeople_machine\nms\new" + "\\" + Fname ,index = 0)                                                            # Save file


<H1> 放入1500 csv </H1>
<h5> Cpath 路徑跟第一個cell切完後的路經相同 </h5>

In [ ]:
Cpath = r"E:\nopeople_machine\felic\new" # first file path
Clist = glob(Cpath + "\\*.csv")         # all first file path csv
CHlist = ["id","x","y","w","h","conf","IDcode"]
count = 0
tcount = 0
for i in range(len(Clist)):
    Ctext = pd.read_csv(Clist[i], header=0, names=CHlist)
    Fname = Clist[i].split("\\")[-1]
    #text = pd.concat([Ftext,Ctext],axis=0)                              # concat two dataframes
    text = Ctext
    text = text.reset_index(drop=True)                                  # reset the index
    cf = Fname.split(".")[0]  
    d,s = countIOU(text,Fname)   
    a = len(d)-len(s)   
    if a<0:
        break         
    count += a
    tcount += len(d)                           # for instance : name = img1001
    print(cf)
    print(d)
    #print(a)                                      # Use the function countIOU(text,Fname) or count(text,Fname) 
    print("---------------------------------------------------------------------------------")
print("del total: " + str(tcount))
print("del diffient: " + str(count))


<h1>畫一張圖測試

In [ ]:
import pandas as pd
from glob import glob
import cv2
import matplotlib.pyplot as plt
imageP = r"E:\nopeople_machine\public\img1075.png"
image = cv2.imread(imageP)
df = pd.read_csv(r"E:\nopeople_machine\nms\new\img1075.csv",header = 0,names = ["id","x","y","w","h","conf","IDcode"])
p = [34,31]
for i in range(len(df["id"])):
        item = [0,0,0,0,0]
        (item[0],item[1],item[2],item[3],item[4]) = ((df["id"][i]),(df["x"][i]),(df["y"][i]),(df["w"][i]),(df["h"][i]))
        
        image_tan = cv2.rectangle(image,(item[1],item[2]),(item[3]+item[1],item[2]+item[4]),(255,0,0),1)
cv2.imwrite("abc.png",image_tan)
#plt.imshow(image_tan)
#plt.show()



<h1> 合併成result.csv</h1>
<h5> file路徑相同第三個cell的存檔路徑  </h5>

In [ ]:
f2 = open(r'E:\nopeople_machine\nms\result.csv','w+') # result csv
files = glob(r"E:\nopeople_machine\nms\new\*.csv")    # new 1500 csv files
for file in files:
  name = file.split("\\")[-1]
  name = name.split(".")[0]                           # for instance : name = img1001
  df = pd.read_csv(file,header = 0)
  for i in range(len(df["id"])):
    f2.write(name + "," + str(df["id"][i]) + "," + str(df["x"][i]) + "," + str(df["y"][i]) + "," + str(df["w"][i]) + "," + str(df["h"][i]) + "\n")

f2.close()